# 测试clip向量化+faiss向量搜索在35类中的分类能力


In [1]:
import random
import time

import torch
from transformers import CLIPProcessor, CLIPModel, CLIPFeatureExtractor

import faiss
from PIL import Image
import os
import json
from collections import Counter
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
with open('./output/id2filename.json', 'r') as json_file:
    id2filename = json.load(json_file)
index = faiss.read_index("./output/image.faiss")

C:\Users\houbenzhou\miniconda3\envs\faiss\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def most_frequent_element(arr):
    if not arr:
        return None  # 如果数组为空，则返回None
    element_count = Counter(arr)
    # 获取出现次数最多的元素和它的次数
    most_common_element, frequency = element_count.most_common(1)[0]
    return most_common_element, frequency

def get_random_image_path(directory):
    # 存储找到的图片路径
    image_paths = []

    # os.walk遍历目录和子目录
    for root, dirs, files in os.walk(directory):
        for file in files:
            # 检查文件是否是图片，这里以几种常见图片格式为例
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                image_paths.append(os.path.join(root, file))

    # 从列表中随机选择一个图片路径
    if image_paths:
        return random.choice(image_paths)
    else:
        return None

def get_image_path(directory):
    # 存储找到的图片路径
    image_paths = []

    # os.walk遍历目录和子目录
    for root, dirs, files in os.walk(directory):
        for file in files:
            # 检查文件是否是图片，这里以几种常见图片格式为例
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                image_paths.append(os.path.join(root, file))

    # 从列表中随机选择一个图片路径
    return image_paths

def get_image_feature(filename: str):
    image = Image.open(filename).convert("RGB")
    processed = processor(images=image, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        image_features = model.get_image_features(pixel_values=processed["pixel_values"])
    return image_features


def image_search(image, k=1):

    inputs = processor(images=image, return_tensors="pt")
    image_features = model.get_image_features(**inputs)
    image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)  # normalize

    image_features = image_features.detach().numpy()
    D, I = index.search(image_features, k)  # 实际的查询

    filenames = [[id2filename[str(j)] for j in i] for i in I]

    return img_path, D, filenames


# 统计正确分类总个数，图片总个数，正确率，按照查询类别最优指标来计算
correct_count: 363 total_count: 387 accuracy: 0.937984496124031

In [6]:
base_images=os.getcwd()
img_path = os.path.join(base_images,"data","新零售图片数据_Trax_部分")
img_paths=get_image_path(img_path)

# 统计正确分类总个数，图片总个数，正确率，按照查询类别最优指标来计算
correct_count = 0
total_count = 0
for img_path in img_paths:
    image = Image.open(img_path)
    start_time = time.time()
    img_path, D, filenames = image_search(image, k=4)
    total_count=total_count+1
    for i in range(len(filenames)):
        if filenames[0][0] == filenames[0][1]:
            correct_count = correct_count + 1
        else:
            print("img_path:", img_path, "true_name:", filenames[0][0], "pred_name:", filenames[0][1])

print("correct_count:",correct_count,"total_count:",total_count,"accuracy:",correct_count/total_count)


img_path: E:\workspaces\2_temp_projects\RealMan_AI_Lab\Recognize-Anything-Box1\Classifer_Anything\latent_features\data\新零售图片数据_Trax_部分\(复)特慧优食品保鲜膜(免刀切抑菌型)30cm宽X150米\1674956159942_1_2016.5_2805.5_333.0_141.0_0.0.jpg true_name: (复)特慧优食品保鲜膜(免刀切抑菌型)30cm宽X150米 pred_name: 5酸甜迷你弹立方山楂金桔酸梅混合味30G
img_path: E:\workspaces\2_temp_projects\RealMan_AI_Lab\Recognize-Anything-Box1\Classifer_Anything\latent_features\data\新零售图片数据_Trax_部分\5酸甜迷你弹立方山楂金桔酸梅混合味30G\1674957227281_2_1361.0_3151.0_248.0_336.0_0.0.jpg true_name: 5酸甜迷你弹立方山楂金桔酸梅混合味30G pred_name: 5酸甜迷你弹立方苹果芒果浆果混合味30G
img_path: E:\workspaces\2_temp_projects\RealMan_AI_Lab\Recognize-Anything-Box1\Classifer_Anything\latent_features\data\新零售图片数据_Trax_部分\5酸甜迷你弹立方山楂金桔酸梅混合味30G\1674957227485_3_1340.5_1163.0_247.0_200.0_0.0.jpg true_name: 5酸甜迷你弹立方山楂金桔酸梅混合味30G pred_name: 5酸甜迷你弹立方苹果芒果浆果混合味30G
img_path: E:\workspaces\2_temp_projects\RealMan_AI_Lab\Recognize-Anything-Box1\Classifer_Anything\latent_features\data\新零售图片数据_Trax_部分\5酸甜迷你弹立方苹果芒果浆果混合味30G\1674957218130_2_

# 统计正确分类总个数，图片总个数，正确率，按照查询类别最优的三个结果中出现次数最多来计算
correct_count: 357 total_count: 387 accuracy: 0.9224806201550387

In [7]:
base_images=os.getcwd()
img_path = os.path.join(base_images,"data","新零售图片数据_Trax_部分")
img_paths=get_image_path(img_path)
correct_count = 0
total_count = 0
for img_path in img_paths:
    image = Image.open(img_path)
    start_time = time.time()
    img_path, D, filenames = image_search(image, k=4)
    temp_filenames=filenames[0]
    true_name=filenames[0][0]
    temp_filenames.pop(0)
    total_count=total_count+1
    pred_name, freq = most_frequent_element(temp_filenames)
    #print("true_name:",true_name,"pred_name:",pred_name,"freq:",freq)
    if true_name == pred_name:
        correct_count = correct_count + 1
    else:
        print("img_path:", img_path, "true_name:", true_name, "pred_name:", pred_name, "freq:", freq)

print("correct_count:",correct_count,"total_count:",total_count,"accuracy:",correct_count/total_count)



img_path: E:\workspaces\2_temp_projects\RealMan_AI_Lab\Recognize-Anything-Box1\Classifer_Anything\latent_features\data\新零售图片数据_Trax_部分\(复)特慧优牛轧糖(蔓越莓味)300G\1674957627602_3_2483.5_1691.0_1053.0_1046.0_0.0.jpg true_name: (复)特慧优牛轧糖(蔓越莓味)300G pred_name: (复)特慧优牛轧糖(抹茶味)300G freq: 2
img_path: E:\workspaces\2_temp_projects\RealMan_AI_Lab\Recognize-Anything-Box1\Classifer_Anything\latent_features\data\新零售图片数据_Trax_部分\(复)特慧优牛轧糖(蔓越莓味)300G\1674957634080_2_2199.5_3544.5_1029.0_991.0_0.0.jpg true_name: (复)特慧优牛轧糖(蔓越莓味)300G pred_name: (复)特慧优牛轧糖(抹茶味)300G freq: 2
img_path: E:\workspaces\2_temp_projects\RealMan_AI_Lab\Recognize-Anything-Box1\Classifer_Anything\latent_features\data\新零售图片数据_Trax_部分\(复)特慧优牛轧糖(蔓越莓味)300G\1674957634497_3_2214.5_1695.0_1041.0_1056.0_0.0.jpg true_name: (复)特慧优牛轧糖(蔓越莓味)300G pred_name: (复)特慧优牛轧糖(抹茶味)300G freq: 2
img_path: E:\workspaces\2_temp_projects\RealMan_AI_Lab\Recognize-Anything-Box1\Classifer_Anything\latent_features\data\新零售图片数据_Trax_部分\(复)特慧优食品保鲜膜(免刀切抑菌型)30cm宽X150米\16749561